In [1]:
from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser

In [3]:
schema = Schema(title=TEXT(stored=True), path=ID(stored=True), content=TEXT)
ix = create_in("indexdir", schema)

with ix.writer() as writer:
    writer.add_document(title=u"First document", content=u"This is the first document we've added!")
    writer.add_document(title=u"Second document", content=u"The second one is even more interesting!")

with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse("interesting")
    results = searcher.search(query)
    print(results)
    for r in results:
        print(r)

<Top 1 Results for Term('content', 'interesting') runtime=0.0003778>
<Hit {'title': 'Second document'}>


In [4]:
print(ix)

FileIndex(FileStorage('indexdir'), 'MAIN')


In [5]:
import pandas as pd
from pandas import Series, DataFrame

In [66]:
sen_docs = pd.read_csv("recent.csv")
sen_docs['hashtags'] = sen_docs.text.str.findall('(#[a-zA-Z]*\w)')
sen_docs['ats'] = sen_docs.text.str.findall('(@[a-zA-Z]*\w)')
sen_docs['url_domains'] = sen_docs.urls.str.findall('://([a-zA-Z]*\w)')

In [70]:
sen_docs.head()

,Unnamed: 0,id,text,time,urls,user,hashtags,ats,url_domains
0,0,1056333751482007553,Americans of all faiths stand with their #Jewi...,1540684566,https://twitter.com/i/web/status/1056333751482...,20015903,[#Jewish],[],[twitter]
1,1,1055940017825214465,"Tonight, families in #Sderot and other communi...",1540590693,https://twitter.com/i/web/status/1055940017825...,20015903,[#Sderot],[],[twitter]
2,2,1055898795282698250,Recently Google suspended accounts associated ...,1540580865,https://twitter.com/i/web/status/1055898795282...,20015903,[],[],[twitter]
3,3,1055552628023676928,My statement on the recent violent actions tak...,1540498332,https://twitter.com/i/web/status/1055552628023...,20015903,[],[],[twitter]
4,4,1053741473630482432,Excited to have been with the Fox River Grove ...,1540066519,https://twitter.com/i/web/status/1053741473630...,20015903,[],[],[twitter]


In [69]:
schema = Schema(
    content=TEXT, 
    id=ID(stored=True),
    hashtags=KEYWORD,
    ats=KEYWORD,
    urls=TEXT
)
ix = create_in("indexdir", schema)

In [71]:
with ix.writer() as writer:
    for doc in sen_docs.iterrows():
        try:
            writer.add_document(
                content=doc[1].text, 
                id=str(doc[1].id),
                hashtags=','.join(doc[1].hashtags),
                ats=','.join(doc[1].ats),
                urls=' '.join(doc[1].urls)
            )
        except:
            pass

In [77]:
search_term = 'cnbc'

with ix.searcher() as searcher:
    query = QueryParser("content", ix.schema).parse(search_term)
    results = searcher.search(query)
    result_ids = []
    for r in results:
        result_ids.append(r['id'])
    results_df = sen_docs[sen_docs.id.isin(result_ids)]
results_df

,Unnamed: 0,id,text,time,urls,user,hashtags,ats,url_domains
175,175,1057711572812996609,"Via @CNBC: Wages and salaries jump by 3.1%, hi...",1541013064,https://www.cnbc.com/2018/10/31/wages-and-sala...,237862972,[],[@CNBC],[www]
205,205,1052263036109635591,Via @CNBC: Another great sign for the economy:...,1539714032,https://www.cnbc.com/2018/10/16/job-openings-h...,237862972,[],[@CNBC],[www]
238,238,1044666690712334337,Via @CNBC: Consumer confidence rises to highes...,1537902922,https://cnb.cx/2xPvvnv,237862972,[],[@CNBC],[cnb]
276,276,1034552480602963971,Via @CNBC: Consumer confidence pops in August ...,1535491507,https://www.cnbc.com/2018/08/28/august-consume...,237862972,[],[@CNBC],[www]


In [13]:
sen_docs = pd.read_csv("recent.csv")